In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/The_Turing_Lens/data_v2
%ls

/content/drive/MyDrive/The_Turing_Lens/data_v2
flagged/                                     resnet50_finetuned_weights_0325_4.pth
resnet50_0327_01_steven.pt                   resnet50_finetuned_weights_0325_5.pth
resnet50_finetuned_weights_0312_output.csv   resnet50_finetuned_weights_0325_6_700.pth
resnet50_finetuned_weights_0312.pth          resnet50_finetuned_weights_0325_6_800_output.csv
resnet50_finetuned_weights_0319_0.pth        resnet50_finetuned_weights_0325_6_800.pth
resnet50_finetuned_weights_0319_1_adamw.pth  resnet50_finetuned_weights_0325_6.pth
resnet50_finetuned_weights_0325_1.pth        test/
resnet50_finetuned_weights_0325_2.pth        train/
resnet50_finetuned_weights_0325_3.pth        val/


In [ ]:
from ipywidgets import interact
from ipywidgets import interact_manual
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(f'計算裝置:{device}')

計算裝置:cpu


In [ ]:
!pip install gradio==4.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 10.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl si

In [ ]:
title_txt = "The Turing Lens 圖靈之眼"
# "Stancode figure Classification with PyTorch Model"
description_txt = "上傳一張人像照，我們將會告訴你這張圖片是真人還是AI圖片"

In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import gradio as gr

import torch
from torchvision.models import resnet50


# 定義模型結構
model = resnet50(num_classes=4)
# 加載權重
model.load_state_dict(torch.load("resnet50_finetuned_weights_0325_6_800.pth", map_location=torch.device(device)))

# 設置模型為評估模式
model.eval()


# 图像预处理
target_size = 500
def preprocess(image):
    preprocess = transform_val = T.Compose([T.Resize((target_size, target_size)), T.ToTensor()])
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0) # 添加批处理维度
    return input_batch

def custom_resize(image):
  transform_train = T.Compose([
    T.Resize(target_size),
    T.ToTensor()
    ])
  # 將圖片應用 transforms
  resized_image = transform_train(image)
  resized_image = resized_image.unsqueeze(0) # 添加批处理维度
  return resized_image

# 模型推理
def predict_image(input_image):
    image = Image.fromarray(input_image.astype('uint8'), 'RGB') # 将NumPy数组转换为PIL图像
    width_orig, height_orig = image.size
    input_tensor = preprocess(image)
    input_tensor_map = custom_resize(image)
    with torch.no_grad():
        output = model(input_tensor)
        probabilities = torch.softmax(output, dim=1)
        probabilities = probabilities.squeeze().numpy()

        # 將機率值轉換為字典形式，方便後續處理
        probabilities_dict = {
            'portrait':probabilities[0],
            'Midjourney':probabilities[1],
            'Stable Diffusion':probabilities[2],
            'Bing':probabilities[3],
        }
        predictions=output.max(1)[1].item()


    ## 開始處理saliency_map
    # 設置requires_grad為True以計算梯度
    input_tensor_map.requires_grad_(True)

    # 計算模型的輸出
    output = model(input_tensor_map)
    pred_class = torch.argmax(output, dim=1)

    # 計算損失，這裡使用對目標類別的分數作為損失
    loss = output[0, pred_class]

    # 計算梯度
    model.zero_grad()
    loss.backward()

    # 從梯度中獲取Saliency map
    saliency_map = input_tensor_map.grad.data.abs().squeeze().numpy()
    saliency_map = np.amax(saliency_map, axis=0)

    # 轉回原始大小
    # transform_back = T.Compose([T.Resize((height_orig, width_orig)), T.ToTensor()])
    # saliency_map = transform_back(saliency_map)

    # 重新轉為圖像檔
    saliency_map_uint8 = (saliency_map * 255).astype(np.uint8)  # 將浮點數陣列轉換為0到255的整數值
    images_aliency_map = Image.fromarray(saliency_map_uint8)

    map_dict = {0:'portrait', 1:'Midjourney', 2:'Stable Diffusion', 3:'Bing'}
    prob_real = probabilities_dict['portrait'] * 100
    prob_ai = (probabilities_dict['Midjourney']+probabilities_dict['Stable Diffusion']+probabilities_dict['Bing']) *100

    if prob_real >= 60:
      ans = f"本圖片有高達{round(prob_real, 1)}%機率來自真實圖片"
      dict_prob_1 = {
          'portrait':prob_real/100,
          'AI image':prob_ai/100
      }
      return images_aliency_map, ans, dict_prob_1, None
    elif prob_real >= 40 and prob_real < 60:
      # ans = f"為AI圖的機率為{round(prob_ai, 2)}%，為真人的機率為{round(prob_real, 2)}%"
      ans = f"此張圖片[為真人]與[為AI]的機率皆超過40%，無法判斷"
      dict_prob_1 = {
          'portrait':prob_real/100,
          'AI image':prob_ai/100
      }
      return images_aliency_map, ans, dict_prob_1, None
    elif prob_real < 40:
      ans = f"本圖片有高達{round(prob_ai, 1)}%機率為AI圖片"
      dict_prob_1 = {
          'portrait':prob_real/100,
          'AI image':prob_ai/100
      }

      dict_prob_2 = {
            'Midjourney':probabilities[1]/(prob_ai/100),
            'Stable Diffusion':probabilities[2]/(prob_ai/100),
            'Bing':probabilities[3]/(prob_ai/100),
        }
      return images_aliency_map, ans, dict_prob_1, dict_prob_2

# 創建輸入組件
input_image = gr.Image(label="Upload Image", height="70%")

# 創建輸出組件
saliency_map_image = gr.Image(label="Saliency map", height="70%")
output_text = gr.Textbox(label="Prediction Result")
label = gr.Label()
label_2 = gr.Label()#num_top_classes=4)

# Theme = gr.Theme.from_hub('step-3-profit/Midnight-Deep')

# 創建介面
gr.Interface(
    fn=predict_image,
    inputs=input_image,
    outputs=[saliency_map_image, output_text, label, label_2],
    title=title_txt,
    description=description_txt,
    theme=gr.Theme.from_hub('bethecloud/storj_theme')
    # theme=Theme # gr.themes.Base()
).launch()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


themes/theme_schema@0.0.1.json:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ac1d2c42d01b728341.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
